In [3]:
with open("./data/Day 24/input.txt") as f:
    data = f.read().splitlines()

with open("./data/Day 24/example.txt") as f:
    example= f.read().splitlines()

In [4]:
from typing import List
def parse_inputs(inputstr: List[str]):
    parsed = []
    for line in inputstr:
        pos, vel = line.split("@")
        pos = tuple(int(x) for x in pos.split(","))
        vel = tuple(int(x) for x in vel.split(","))
        parsed.append((pos, vel))
    return parsed

In [5]:
def crosses(hail1, hail2, minimum, maximum):
    x_1, y_1, _ = hail1[0]
    x_2, y_2, _ = hail2[0]
    v_x1, v_y1, _ = hail1[1]
    v_x2, v_y2, _ = hail2[1]
    try:
        x = (-y_1 + y_2 + ((v_y1 / v_x1) * x_1) - ((v_y2 / v_x2) * x_2)) / ((v_y1 / v_x1) - (v_y2 / v_x2))
    except ZeroDivisionError:
        return False
    ta = (x - x_1) / v_x1
    tb = (x - x_2) / v_x2
    y = y_1 + v_y1 * ta
    if ta < 0 or tb < 0:
        return False
    if minimum <= x <= maximum and minimum <= y <= maximum:
        return True
    return False

# Part 1

In [6]:
checks = []
example_hail = parse_inputs(example)
for i, item1 in enumerate(example_hail):
    for j, item2 in enumerate(example_hail[i+1:]):
        checks.append(crosses(item1, item2, 7, 27))
sum(checks)

2

In [7]:
checks = []
hail = parse_inputs(data)
for i, item1 in enumerate(hail):
    for j, item2 in enumerate(hail[i + 1 :]):
        checks.append(crosses(item1, item2, 200000000000000, 400000000000000))
sum(checks)

15107

# Part 2

In [8]:
import z3

In [9]:
def solve_stones(hail: list[tuple[int]]):
	# BitVec is way faster than Int
	I = lambda name: z3.BitVec(name, 64)
	solver = z3.Solver()
	x, y, z = I("x"), I("y"), I("z")
	vx, vy, vz = I("vx"), I("vy"), I("vz")
	for i, a in enumerate(hail):
		pos, speed = a
		vax, vay, vaz = speed
		ax, ay, az = pos
		t = I(f't_{i}')
		solver.add(t >= 0)
		solver.add(x + vx * t == ax + vax * t)
		solver.add(y + vy * t == ay + vay * t)
		solver.add(z + vz * t == az + vaz * t)	
	assert solver.check() == z3.sat
	m = solver.model()
	x, y, z = m.eval(x), m.eval(y), m.eval(z)#, m.eval(vx), m.eval(vy), m.eval(vz)
	x, y, z = x.as_long(), y.as_long(), z.as_long()#, vx.as_long(), vy.as_long(), vz.as_long()
	return x, y, z#, vx, vy, vz

In [12]:
example_x, example_y, example_z = solve_stones(example_hail)

In [13]:
print(f"Position: {example_x}, {example_y}, {example_z}")

Position: 24, 13, 10


In [10]:
x, y, z = solve_stones(hail)

In [11]:
print(f"Position: {x}, {y}, {z}")
print(f"Distance: {x + y + z}")

Position: 422644646660238, 244357651988392, 189640099899118
Distance: 856642398547748
